# RidgeClassifier

In [1]:
import os
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline 
import gensim
from gensim import corpora,models,similarities
from gensim.models import Word2Vec
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
df=pd.read_csv("Mock_data_combined.csv",encoding="unicode_escape")
df.head()
df1=df[["Code Chunck","Indicator"]]
df1["Indicator"]=df['Indicator'].astype(int)

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [3]:
def ridge_clf():
    # df.columns.tolist()
    #Text Normalization
    tokenized_case = df1['Code Chunck'].apply(lambda x: x.split())
    bow_vectorizer = CountVectorizer(max_df =0.90,min_df=2,max_features=1000,stop_words='english')
    bow=bow_vectorizer.fit_transform(df1['Code Chunck'])
#     bow.shape
    tfidf_vectorizer = TfidfVectorizer(max_df =0.90,min_df=2,max_features=1000,stop_words='english')
    tfidf = tfidf_vectorizer.fit_transform(df1['Code Chunck'])
    wordvec_arrays=np.zeros((len(tokenized_case),100))
    for i in range(len(tokenized_case)):
        wordvec_arrays[i,:]=word_vector(tokenized_case[i],100)
        wordvec_df = pd.DataFrame(wordvec_arrays)
    print(wordvec_df.shape)
    return wordvec_df

In [4]:
def word_vector(tokens,size):
    tokenized_case = df1['Code Chunck'].apply(lambda x: x.split())#tokenizing
    model_w2v = gensim.models.Word2Vec(tokenized_case,size=100,window=5,min_count=2,sg=1,hs=0,negative=10,workers=2,seed=34)
    model_w2v.train(tokenized_case,total_examples = len(df1['Code Chunck']),epochs=20)
#     model_w2v.wv.most_similar(positive='assert')
#     len(model_w2v['assert'])
    vec= np.zeros(size).reshape(1,size)
    count=0.
    for word in tokens:
        try:
            vec +=model_w2v[word].reshape((1,size))
            count+=1.
        except KeyError:
            continue
    if count!=0:
        vec/=count
    return vec      

In [5]:
wordvec_df=ridge_clf()
train_df=wordvec_df
x=train_df
target_df=df1["Indicator"]
y=target_df

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


(50, 100)


In [6]:
x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x, y, test_size = 0.5)
# print(x_training_data)
# print(x_test_data)
# print(y_training_data)
print(y_test_data)

32    0
31    1
40    0
17    1
5     1
7     1
45    0
36    0
13    0
9     0
15    0
2     0
22    0
41    1
6     1
27    1
38    1
16    1
39    1
26    0
3     1
11    0
46    1
25    1
35    1
Name: Indicator, dtype: int32


# RidgeClassifier 

In [7]:
# # make a prediction with a ridge regression model on the dataset
# from pandas import read_csv
# from sklearn.linear_model import Ridge
# from sklearn.linear_model import Ridge
# import numpy as np
# from sklearn.linear_model import Ridge

# ## training the model

# ridgeReg = Ridge(alpha=0.05, normalize=True)

# ridgeReg.fit(x_training_data,y_training_data)

# pred = ridgeReg.predict(x_test_data)
# print(pred)

# # calculating mse

# mse = np.mean((pred - y_test_data)**2)
# # mse

In [8]:
from sklearn.linear_model import RidgeClassifier
clf = RidgeClassifier().fit(x, y)
pred = clf.predict(x_test_data)
pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1])

In [9]:
yprd_df=pd.DataFrame(pred,index=list(x_test_data.index.values))
yprd_df=yprd_df.rename(columns={0:"Ridge_Predicted_Indicator"})
yprd_df

,Ridge_Predicted_Indicator
32,1
31,1
40,1
17,1
5,1
7,1
45,1
36,1
13,1
9,1


In [10]:
# df1=df1.join(yprd_df)
# df1.to_csv("prediction.csv",index=False)

final_df=df1.iloc[list(x_test_data.index.values),:]
final_df=pd.concat([final_df,yprd_df],axis=1)
final_df
final_df.to_csv("Ridge_prediction.csv",index=False)

In [11]:
# calculating score 
clf.score(x, y)

0.52